In [ ]:
# Before run this code, select the GPU of Runtime.
if(!require(torch)){install.packages("torch", repos = "http://cran.us.r-project.org")}
if(!require(tidyverse)) install.packages("tidyverse", repos = "http://cran.us.r-project.org")
if(!require(caret)) install.packages("caret", repos = "http://cran.us.r-project.org")
if(!require(tidymodels)){install.packages("tidymodels", repos = "http://cran.us.r-project.org")}


Loading required package: torch

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘torch’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘coro’


Loading required package: tidyverse

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.7
✔ tidyr   1.1.4     ✔ stringr 1.4.0
✔ readr   2.1.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Loading required package: caret

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘caret’”
Instal

In [ ]:

library(torch)
library(tidyverse)
library(caret)
library(tidymodels)

# Wine Quality Data Set
# https://archive.ics.uci.edu/ml/datasets/Wine+Quality
# https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv

url <- c("https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv")
wine <- read_delim(url,delim = ";")

# modify the column name

colnames(wine) <- c("fixed_acidity", "volatile_acidity","citric_acid",
                    "residual_sugar","chlorides","free_sulfur_dioxide",
                    "total_sulfur_dioxide","density","pH",
                    "sulphates","alcohol","quality")

set.seed(1, sample.kind="Rounding")
test_index <- createDataPartition(y = wine$quality, times = 1, p = 0.2, list = FALSE)

### Data preparation: Split the data set ----
# Preparation test and train data set for model selection
# Due to the small amount of data, 20% was used as test data.
set.seed(1, sample.kind="Rounding")
test_index <- createDataPartition(y = wine$quality, times = 1, p = 0.2, list = FALSE)
train_set <- wine[-test_index,]
test_set <- wine[test_index,]

# To build each model, centralization and standardization are done.
rec_pre <- recipe(train_set, quality~.) %>% 
  step_center(all_numeric(),-quality) %>% # Centralization 
  step_scale(all_numeric(), -quality) %>%  # Standadization
  # step_knnimpute(all_predictors(),K=5)  # There is no NA
  # step_dummy(all_predictors(), -all_numeric()) # There is no categorical data
  prep()

# apply the recipe
train_torch <- bake(rec_pre, new_data = train_set)
test_torch <-  bake(rec_pre, new_data = test_set)

# Transform "quality" into binary
train_torch$quality[train_torch$quality < 6] <- 0
test_torch$quality[test_torch$quality < 6] <- 0
train_torch$quality[train_torch$quality > 0] <- 1
test_torch$quality[test_torch$quality > 0] <- 1

# Create {torch} dataset

df_dataset <- dataset(
  
  "wine",
  
  initialize = function(df, response_variable) {
    self$df <- df[,-which(names(df) == response_variable)]
    self$response_variable <- df[[response_variable]]
  },
  
  .getitem = function(index) {
    response <- torch_tensor(self$response_variable[index])
    x <- torch_tensor(as.numeric(self$df[index,]))
    
    list(x = x, y = response)
  },
  
  .length = function() {
    length(self$response_variable)
  }
  
)

train_torch_ds <- df_dataset(train_torch, "quality")
test_torch_ds <- df_dataset(test_torch, "quality")

train_torch_dl <- dataloader(train_torch_ds, batch_size = 32, shuffle = T)
test_torch_dl <- dataloader(test_torch_ds, batch_size = 1, shuffle = F)

# Define a network

net <- nn_module(
  
  "wine_DNN",
  
  initialize = function() {
    self$fc1 <- nn_linear(11, 66)
    self$fc2 <- nn_linear(66, 44)
    self$fc3 <- nn_linear(44, 1)
    self$dropout <- nn_dropout(0.5)
  },
  
  forward = function(x) {
    x %>% 
      self$fc1() %>%
      nnf_relu() %>%
      self$fc2() %>%
      nnf_relu() %>%
      self$dropout() %>%
      self$fc3() %>%
      nnf_sigmoid()
  }
)


Loading required package: lattice


Attaching package: ‘caret’


The following object is masked from ‘package:purrr’:

    lift


Registered S3 method overwritten by 'tune':
  method                   from   
  required_pkgs.model_spec parsnip

── Attaching packages ────────────────────────────────────── tidymodels 0.1.4 ──

✔ broom        0.7.10     ✔ rsample      0.1.1 
✔ dials        0.0.10     ✔ tune         0.1.6 
✔ infer        1.0.0      ✔ workflows    0.2.4 
✔ modeldata    0.1.1      ✔ workflowsets 0.1.0 
✔ parsnip      0.1.7      ✔ yardstick    0.0.9 
✔ recipes      0.1.17     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard()        masks purrr::discard()
✖ dplyr::filter()          masks stats::filter()
✖ recipes::fixed()         masks stringr::fixed()
✖ dplyr::lag()             masks stats::lag()
✖ caret::lift()            masks purrr::lift()
✖ yardstick::precision()   masks caret::precision()
✖ yardstick::recall()      mas

In [ ]:
# CPU
startTime = Sys.time()

model <- net()
model$to(device = "cpu")

optimizer <- optim_adam(model$parameters,lr = 0.01)

# Run a learning iteration

coro::loop(for (epoch in 1:2000) {
  
  l <- c()
  
  coro::loop(for (b in enumerate(train_torch_dl)) {
    optimizer$zero_grad()
    output <- model(b[[1]]$to(device = "cpu"))
    loss <- nnf_binary_cross_entropy_with_logits(output, b[[2]]$to(device = "cpu"))
    loss$backward()
    optimizer$step()
    l <- c(l, loss$item())
  })
  
  cat(sprintf("Loss at epoch %d: %3f\n", epoch, mean(l)))
})


model$eval()

i <- 1
pred_labels <- rep(0, nrow(test_torch))

coro::loop(for (b in enumerate(test_torch_dl)) {
  output <- model(b[[1]]$to(device = "cpu"))
  pred_labels[i] <- round(output$item(), 0)
  i <- i + 1
})

# Evaluate

table(test=test_torch$quality, pred_labels)

(torch_acc <- sum(diag(table(test_torch$quality, pred_labels))) / nrow(test_torch))

endTime = Sys.time() 

print(endTime - startTime) 

Warning message:
“The `enumerate` construct is deprecated in favor of the `coro::loop` syntax.
* See https://github.com/mlverse/torch/issues/558 for more information.”


Loss at epoch 1: 0.632029


Warning message:
“The `enumerate` construct is deprecated in favor of the `coro::loop` syntax.
* See https://github.com/mlverse/torch/issues/558 for more information.”


Loss at epoch 2: 0.608550


Warning message:
“The `enumerate` construct is deprecated in favor of the `coro::loop` syntax.
* See https://github.com/mlverse/torch/issues/558 for more information.”


...

Loss at epoch 2000: 0.560567


Warning message:
“The `enumerate` construct is deprecated in favor of the `coro::loop` syntax.
* See https://github.com/mlverse/torch/issues/558 for more information.”


    pred_labels
test   0   1
   0 117  32
   1  51 121

[1] 0.741433

Time difference of 53.83597 mins


In [ ]:
# GPU
startTime = Sys.time()

model <- net()
model$to(device = "cuda")

optimizer <- optim_adam(model$parameters,lr = 0.01)

# Run a learning iteration

coro::loop(for (epoch in 1:2000) {
  
  l <- c()
  
  coro::loop(for (b in enumerate(train_torch_dl)) {
    optimizer$zero_grad()
    output <- model(b[[1]]$to(device = "cuda"))
    loss <- nnf_binary_cross_entropy_with_logits(output, b[[2]]$to(device = "cuda"))
    loss$backward()
    optimizer$step()
    l <- c(l, loss$item())
  })
  
  cat(sprintf("Loss at epoch %d: %3f\n", epoch, mean(l)))
})


model$eval()

i <- 1
pred_labels <- rep(0, nrow(test_torch))

coro::loop(for (b in enumerate(test_torch_dl)) {
  output <- model(b[[1]]$to(device = "cuda"))
  pred_labels[i] <- round(output$item(), 0)
  i <- i + 1
})

# Evaluate

table(test=test_torch$quality, pred_labels)

(torch_acc <- sum(diag(table(test_torch$quality, pred_labels))) / nrow(test_torch))

endTime = Sys.time() 

print(endTime - startTime) 

Warning message:
“The `enumerate` construct is deprecated in favor of the `coro::loop` syntax.
* See https://github.com/mlverse/torch/issues/558 for more information.”


Loss at epoch 1: 0.634672


Warning message:
“The `enumerate` construct is deprecated in favor of the `coro::loop` syntax.
* See https://github.com/mlverse/torch/issues/558 for more information.”


Loss at epoch 2: 0.609743


Warning message:
“The `enumerate` construct is deprecated in favor of the `coro::loop` syntax.
* See https://github.com/mlverse/torch/issues/558 for more information.”


...

Loss at epoch 2000: 0.575787


Warning message:
“The `enumerate` construct is deprecated in favor of the `coro::loop` syntax.
* See https://github.com/mlverse/torch/issues/558 for more information.”


    pred_labels
test   0   1
   0 114  35
   1  54 118

[1] 0.7227414

Time difference of 35.2032 mins
